### Zero sum game, solve as LP

Given a game zero sum game, player $ A $ wins by the amount player $ B $ loses. This is the payoff matrix for $ A $:

![player A payoff matrix](Screenshot_2023-08-03_08-55-14.png)

### Pure strategy

In game theory, a rational decision is to play it safe - consider all of the bad options and try to find the best of those.

For $ A $ this will be max-min (functional order of min then max). So for each row, which is the worst result, and add those to a vector. Find the best of those.

For $ B $ this will be min-max (functional order of max then min). Each positive value is a bad result for $ B $, so it must consider all of the bad results. So it maximises each column (max) and then takes the min of those.

### Mixed strategy

Instead of selecting a single action, we can instead assign probabilities of for the actions to lead to the optimum.

Also, since we're trying to find probabilities, we should ensure that $ x = 1 $

_Below we will see that the programs for $ A, B $ are dual programs of each other._

__LP program for $ A $ is to formulate this as a minimisation problem, with some unknown $ \alpha $, and then transform it into a maximisation problem__

Note that below, the $ 0 $ payoff is not needed, but we would add it to the matrix to be optimised in Matlab.

$ \left\{ \begin{matrix} \max \alpha \\ 
x_2 - x_3 \ge \alpha \\
3x_1 - x_2 + x_3 \ge \alpha \\
x_1 + x_2 + x_3 = 1 \\
x_1, x_2, x_3 \ge 0 \end{matrix} \right. $

But this doesn't yet have the form of a maximisation program. We should switch $ \alpha $

$ \left\{ \begin{matrix} \max \alpha \\ 
- x_2 + x_3 + \alpha \le 0 \\
- 3x_1 + x_2 - x_3 + \alpha \le 0 \\
x_1 + x_2 + x_3 = 1 \\
x_1, x_2, x_3 \ge 0 \end{matrix} \right. $

__LP program for $ B $ is to formulate this as a maximisation problem, with some unknown $ \beta $, and then transform it into a minimisation problem__

Notice that since this is a min-max, we write the constraints in the maximisation form, since for $ B $ these are the bad results.

$ \left\{ \begin{matrix} \min \beta \\ 
3y_2 \le \beta \\
y_1 - y_2 \le \beta \\
- 2y_1 + y_2 \le \beta \\
y_1 + y_2 = 1 \\
y_1, y_2 \ge 0 \end{matrix} \right. $

Then we must represent this in its minimisation form:

$ \left\{ \begin{matrix} \min \beta \\ 
-3y_2 -\beta \ge 0 \\
-y_1 + y_2 -\beta \ge 0 \\
2y_1 - y_2 -\beta \ge 0 \\
y_1 + y_2 = 1 \\
y_1, y_2 \ge 0 \end{matrix} \right. $

In [30]:
### Player A LP in Gnu Octave

# Need to find x=(x_1, x_2, x_3, \alpha)

f = [0, 0, 0, 1];

A = [ 0, -1,  2, 1;
     -3,  1, -1, 1;
      1,  1,  1, 0 ];
     
b = [0;0;1];
lb = [0;0;0;0];

param.dual = 3;

ctype = "UUS";
vartype = "CCCC"; # continuous variable, usually you don't want an integer variable
sense = -1;

[x, fmin, status, extra] = glpk(f, A, b, lb, [], ctype, vartype, sense, param);

% To check duality, do
% Note, that since LP is a max problem and the Dual is a min problem, take
% the oposite of one of them
acceptableError = 10^(-4);
objFunctionDifference = transpose(extra.lambda)*b - f*x;
try
    assert(abs(objFunctionDifference) <= acceptableError);
catch
    error("difference of LP and Dual objective functions outside acceptable error");
end

format rat

# Optimal decision variables are:
disp("FOR A")
disp(x)

disp("FOR B")
disp(extra.lambda)


FOR A
        2/5
        3/5
          0
        3/5
FOR B
        4/5
        1/5
        3/5


In [31]:
clear;

options=optimoptions(@linprog, "Algorithm", "dual-simplex");

f=[0, 0, 0, -1]; % invert these for max problem

% inequality constraints
A=[ 0,-1, 2, 1;
   -3, 1,-1, 1];
b = [0;
     0];

% equality constraints
A_eq = [1, 1, 1, 0];
b_eq = [1];

lb = [0;0;0;0];

[x, fval, exitflag, output, lambda] = linprog(f,A,b,A_eq,b_eq,lb,[],options);

% To check duality, do
% Note, that since LP is a max problem and the Dual is a min problem, take
% the oposite of one of them

lambda_combined = [lambda.ineqlin;lambda.eqlin];
b_combined = [b;b_eq];


acceptableError = 10^(-4);
objFunctionDifference = -transpose(lambda_combined)*b_combined - f*x;

try
    assert(abs(objFunctionDifference) <= acceptableError);
catch
    error("difference of LP and Dual objective functions outside acceptable error");
end

format rat

% Optimal decision values are:
disp("FOR A");
disp(x);

disp("FOR B");
disp(lambda.ineqlin);


error: 'optimoptions' undefined near line 1, column 1

The 'optimoptions' function belongs to the optim package from Octave
Forge which you have installed but not loaded.  To load the package, run
'pkg load optim' from the Octave prompt.

Please read <https://www.octave.org/missing.html> to learn how you can
contribute missing functionality.
error: 'options' undefined near line 1, column 1
error: 'lambda' undefined near line 1, column 1
error: 'lambda_combined' undefined near line 1, column 1
error: difference of LP and Dual objective functions outside acceptable error
FOR A
error: 'x' undefined near line 1, column 1
FOR B
error: 'lambda' undefined near line 1, column 1
